In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sklearn 
from sklearn import preprocessing
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import plot_confusion_matrix, roc_auc_score, plot_roc_curve, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import optuna

In [ ]:
input_path=Path('/kaggle/input/tabular-playground-series-mar-2021/')

In [ ]:
train=pd.read_csv(input_path / 'train.csv')
test=pd.read_csv(input_path / 'test.csv')
submission=pd.read_csv(input_path / 'sample_submission.csv')

In [ ]:
train.columns

In [ ]:
train=train.drop(['cat10'], axis=1)
test=test.drop(['cat10'], axis=1)

In [ ]:
column=[col for col in train.columns if col != ('id' and 'target')]
    

In [ ]:
for cols in train.columns:
    if train[cols].dtype =='object':
        lbl=LabelEncoder()
        lbl.fit(list(train[cols].values) + list(test[cols].values))
        train[cols]=lbl.transform(train[cols].values)
        test[cols]=lbl.transform(test[cols].values)
        

In [ ]:
train.head()

In [ ]:
X=train.drop(["id", "target"], axis=1)
y=train.target

In [ ]:
X.shape, y.shape

In [ ]:
def K_FOLD(X, y, model, params, folds=9):
    roc_score=[]
    skf=StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    for fold , (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f'Fold: {fold}')
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test  = X.iloc[test_idx], y.iloc[test_idx]
        
        clf=model(**params)
        clf.fit(X_train,y_train, eval_set=[(X_test, y_test)],
                early_stopping_rounds=100,
                verbose=False)
        score=roc_auc_score(y_test, clf.predict_proba(X_test)[:,-1])
        roc_score.append(score)
        print(f"ROC AUC Score: {score}")
        print("Name")
        print("-"*25)
        
        
    return clf, np.mean(roc_score)     

In [ ]:
param = {
    'metric': 'auc', 
    'reg_alpha': 6.010538011450937, 
    'reg_lambda': 0.031702113663443346, 
    'colsample_bytree': 0.27,
    'subsample': 0.6, 
    'learning_rate': 0.05, 
    'max_depth': 100, 
    'num_leaves': 100, 
    'min_child_samples': 216,
    'cat_smooth': 87, 
    'random_state': 48,
    'n_estimators': 20000
}


In [ ]:
best_params = {
    'learning_rate': 0.03,
     'metric': 'auc',
     'n_estimators': 621,
     'num_leaves': 20067,
     'max_depth': 27,
     'reg_alpha': 9.630576598001266,
     'reg_lambda': 2.346945113164939,
     'colsample_bytree': 0.29858836720777177,
     'subsample ': 0.6,
     'cat_smooth': 20.45469933468782
}

In [ ]:
clf, score = K_FOLD(X, y, LGBMClassifier, best_params, 9)

In [ ]:
xgb_param = {
    "seed":50,
    "n_estimators":10000,
    "verbosity":1,
    "eval_metric":"auc",
    "gpu_id":0,
    "alpha":9.037672745139417,
    "colsample_bytree":0.6204453741210664,
    "gamma":0.7655610995827371,
    "reg_lambda":6.854931929134254,
    "learning_rate":0.013401479391378243,
    "max_bin":304,
    "max_depth":14,
    "min_child_weight":1.5513425169835457,
    "subsample":0.8303017072175757,
}

In [ ]:
clf_xgb, xgb_score = K_fold(X, y, XGBClassifier, xgb_param, 10)

In [ ]:
cb_param = {
    "verbose":0,
    "eval_metric":"AUC",
    "loss_function":"Logloss",
    "random_state":50,
    "num_boost_round":20000,
    "od_type":"Iter",
    "od_wait":200,
    "devices":"0",
    "bagging_temperature":1.288692494969795,
    "grow_policy":"Depthwise",
    "l2_leaf_reg":9.847870133539244,
    "learning_rate":0.01877982653902465,
    "max_depth":8,
    "min_data_in_leaf":1,
    "penalties_coefficient":2.1176668909602734,
}

In [ ]:
clf_cb, cb_score = K_FOLD(X, y, CatBoostClassifier, cb_param, 10)

In [ ]:
# LGB 
submission["target"] = clf.predict_proba(test[X.columns])[:, 1]
submission.to_csv("Lightgbm.csv", index=False)


submission["target"] = clf_cb.predict_proba(test[X.columns])[:, 1]
submission.to_csv("Catboost.csv", index=False)